# <center><h1>Lippmann-Schwinger and phase-shift</h1><center>

In [1]:
%run potential_num.ipynb

from numpy.linalg import solve, det, inv
from numpy import matmul as mul

<IPython.core.display.Javascript object>

In [2]:
# # Example

# phase_list=phase_dict[0]
# p_wave = [phase_name(*phase) for phase in phase_list]
# print(p_wave)

# q0 = 100 # MeV
# lam = 52 # fm^-1
# c0 = 2.2

# Vtensor0 = Vtensor_calc(phase_list)
# print('Vtensor0')
# print_(Vtensor0)

# print('\n\nVtensor2')
# Vtensor_q0 = Vtensor_q0_calc(phase_list, Vtensor0, q0)
# print_(Vtensor_q0)

# print('\n\nVtensor')
# Vtensor = pot_lambda(Vtensor_q0,c0,lam)

# print_(Vtensor)

## $$\sum_{\alpha''}\sum_k A_{jk}^{\alpha'\alpha''}t_{ki}^{\alpha''\alpha'}=V_{ji}^{\alpha'\alpha}$$

In [3]:
def A_matrix_cal(s1, l1, j1, s2, l2, j2, Vtensor):

    def I_lambda():

        return  ( 
                    (2*pgrid[Np])**(-1)
                    * log( (pgrid[Np] + p3)/ (p3 - pgrid[Np]) )
                )
    
    ans1 = 0

    for ip in range(Np):

        ans1 += (pgrid[Np]**2 * wp[ip])/( pgrid[Np]**2 - pgrid[ip]**2 )

    #   a(j,k)
    
    def ajk(s1, l1, j1, s2, l2, j2,  j, k):

        return  ( 
                    kron(j,k)
                    * kron(l1,l2)  * kron(s1,s2) * kron(j1,j2) 

                    -

                    (
                        2 * mu 
                        * Vtensor[s1, l1, j1, s2, l2, j2, j, k]
                        * pgrid[k]**2 * wp[k]
                    )/(pgrid[Np]**2 - pgrid[k]**2)
            )

    

    #   a(j,Np)
    
    def ajNp(s1, l1, j1, s2, l2, j2, j):   

        return  (
                    2 * mu
                    * ans1 
                    * Vtensor[s1, l1, j1, s2, l2, j2, j, Np]

                    +

                    (1j * pi * mu * pgrid[Np] - 2 * mu * pgrid[Np]**2 *  I_lambda())
                    * Vtensor[s1, l1, j1, s2, l2, j2, j, Np]
                )
    
    #   a(Np,k)
    
    def aNpk(s1, l1, j1, s2, l2, j2, k):

        return  (
                    - 2 * mu * ( (wp[k] * pgrid[k]**2)/(pgrid[Np]**2 - pgrid[k]**2) * \
                            Vtensor[s1, l1, j1, s2, l2, j2, Np, k])
                )
    
    #   a(Np,Np)

    def aNpNp(s1, l1, j1, s2, l2, j2):   

        return  (
                    kron(l1,l2)    * kron(s1,s2) * kron(j1,j2)

                    +

                    2 * mu * ans1
                    * Vtensor[s1, l1, j1, s2, l2, j2, Np, Np]

                    +

                    ( 1j * pi * mu * pgrid[Np] - 2 * mu * pgrid[Np]**2 * I_lambda())
                    * Vtensor[s1, l1, j1, s2, l2, j2, Np, Np]
                )

    Atensor={}

    for j_ in range(Np):
        for k_ in range(Np):
            Atensor[s1, l1, j1, s2, l2, j2, j_, k_ ] = ajk(s1, l1, j1, s2, l2, j2, j_, k_)
            

    for j_ in range(Np):

        Atensor[s1, l1, j1, s2, l2, j2, j_, Np ] = ajNp(s1, l1, j1, s2, l2, j2, j_)
        Atensor[s1, l1, j1, s2, l2, j2, Np, j_ ] = aNpk(s1, l1, j1, s2, l2, j2, j_)

    Atensor[s1, l1, j1, s2, l2, j2, Np, Np] = aNpNp(s1, l1, j1, s2, l2, j2)

    return Atensor

In [4]:
# # Example

# print(p_wave)
# s1, l1, j1 = 0,0,0
# s2, l2, j2 = 0,0,0
# Amat = A_matrix_cal(s1, l1, j1, s2, l2, j2, Vtensor)

# print_(Amat)
# print('\n\nnumber of channels in Vtensor  : ',len(Vtensor)/((Np+1)**2))
# print('number of channels in Amat     : ',len(Amat)/((Np+1)**2))

## Phase shift: $S=e^{2i\delta}$

In [5]:
def phase_shift(s_mat):
    
    def delta_phase(S):
        
        sx = S.real/abs(S)
        sy = S.imag/abs(S)

        delta = 0

        if (sx>0 and sy>0):
            delta = asin(sy)/2.*(180./pi)
        elif (sx<0 and sy>0):
            delta =  90 - asin(sy)/2.*(180./pi)
        elif (sx<0 and sy<0):
            delta = 90 - asin(sy)/2.*(180./pi) 
        elif (sx>0 and sy<0):
            delta = asin(sy)/2.*(180./pi)
        elif (sx==0 and sy==0):
            delta = 0
        elif sx==0:
            if sy>0:
                delta = pi/2
            else:
                delta = 3*pi/2
        elif sy==0:
            if sx>0:
                delta = 0
            else:
                delta = pi
        else:
            print("recheck the phase shift code.\n")

        return delta
    
    if s_mat.shape == (1,1):
        
        return [delta_phase(s_mat[0,0])]
    
    elif s_mat.shape == (2,2):

        s00 = s_mat[0,0]
        s11 = s_mat[1,1]

        e0  = acos( abs(s00) )/2 
        e0  = e0 * 180/pi
        
        d0  = delta_phase(s00)
        d2  = delta_phase(s11)

        return [d0,d2,e0]

In [6]:
# # Example 

# d0 = -20
# s = np.matrix([exp(2j * d0 * pi/180)])

# list1 = phase_shift(s)


# print(list1)

In [8]:
# # Example

# d0, d2, e0 = -5, -100, 15

# print('d0 = {}, d2 = {}, e0 = {}\n'.format(d0,d2,e0))

# exp_del0 = exp(2j * d0 * pi/180) #  =phase_shift
# exp_del2 =exp(2j * d2 * pi/180)
# eps = 2 * e0 * pi/180

# s_mat = np.matrix([[exp_del0 * cos(eps),1j * np.sqrt(exp_del0 * exp_del2) * sin(eps)],
#                   [1j * np.sqrt(exp_del0 * exp_del2) * sin(eps),exp_del2* cos(eps)]])
# print('s.s_dagger:')

# s_sd = np.matmul(s_mat.getH(),s_mat)

# print(round_n(s_sd))
# print('\nPhase shift:')
# print(phase_shift(s_mat))

# del d0, d2, e0, exp_del0, exp_del2,eps,s_mat, s_sd

## Phase shift for a given $q_0, C_0, \Lambda$

----
### uncoupled phase: $\{\alpha\}$

### Note: Since uncoupled phase is coupled phase of length 1, there is no need for uncoupled phase shift calculator, except for testing

In [8]:
# def phase_shift_calc_uncoupled(s1, l1, j1, s2, l2, j2, c0, q0, Vtensor, lam):

#     Vtensor1 =  pot_lambda(Vtensor,c0,lam)

#     Atensor = A_matrix_cal(s1, l1, j1, s2, l2, j2, Vtensor1)

#     Vjk = np.matrix(np.empty([ Np+1, Np+1], dtype = complex))
#     Ajk = np.matrix(np.empty([ Np+1, Np+1], dtype = complex))

#     for j_ in range(Np+1):
#         for k_ in range(Np+1):
#             Vjk[j_,k_] = Vtensor1[s1, l1, j1, s2, l2, j2, j_, k_]
#             Ajk[j_,k_] = Atensor[s1, l1, j1, s2, l2, j2, j_, k_]

#     t_matrix = solve(Ajk,Vjk)

#     s_mat = 1 - 1j * pi * m_n * q0 * t_matrix[Np,Np]

#     delta = phase_shift(s_mat)

#     return [delta, s_mat]

In [9]:
# #Example

# [s1,l1,j1] = phase_list[0]
# [s2,l2,j2] = phase_list[0]

# print(round_n(phase_shift_calc_uncoupled(s1, l1, j1, s2, l2, j2, c0, q0, Vtensor, lam)),'\n')


### coupled phase: $\{\alpha_1,\alpha_2,\ldots\}$

In [10]:
def phase_shift_calc(phase_list, c0, q0, Vtensor, lam):

       
    n = Np + 1
    len_phase = len(phase_list)

    # create empty V and A matrices
    
    Vjk = np.matrix(np.empty([ n * len_phase, n * len_phase], dtype = complex)) 
    Ajk = np.matrix(np.empty([ n * len_phase, n * len_phase], dtype = complex))

    # empty S-matrix
    s_mat = np.matrix(np.empty([ len_phase, len_phase], dtype = complex))
    
    # introducing the regulator and adding the LO contact term
    Vtensor1 = pot_lambda(Vtensor,c0,lam)

    for j_1 in range(len_phase):
        for k_1 in range(len_phase):

            [s1,l1,j1] = phase_list[j_1]
            [s2,l2,j2] = phase_list[k_1]

            Atensor = A_matrix_cal(s1, l1, j1, s2, l2, j2, Vtensor1)

            for j_ in range(n):
                for k_ in range(n):

                    Vjk[j_1*n + j_,k_1*n + k_] = Vtensor1[s1, l1, j1, s2, l2, j2, j_, k_ ]
                    Ajk[j_1*n + j_,k_1*n + k_] =  Atensor[s1, l1, j1, s2, l2, j2, j_, k_ ]
                    
    # solving T = A.V
    t_matrix = solve(Ajk,Vjk)
    
    # for uncoupled channel
    if len_phase == 1:
        s_mat = np.matrix(1 - 1j * pi * m_n * q0 * t_matrix[Np,Np])

        [delta] = phase_shift(s_mat)

        return [delta, s_mat]
    
    # for cp-even coupled channel
    elif len_phase == 2:
        for j_ in range(len_phase):
            for k_ in range(len_phase):

                [s1,l1,j1] = phase_list[j_]
                [s2,l2,j2] = phase_list[k_]

                s_mat[j_,k_] = ( 
                                    equal(s1,s2) * equal(l1,l2) 
                                   - 1j * pi * m_n * q0 * t_matrix[j_*n + Np,k_*n + Np]
                                  )

        [d0,d2,e0]  = phase_shift(s_mat)

        return [d0,d2,e0, s_mat]

In [11]:
# # Example

# phase_list = [[0,0,0]]
# p_wave = [phase_name(*phase) for phase in phase_list]

# print('phases :',phase_list,'==',p_wave)

# lam = 2
# c0 = -45.39767
# q0 = sqrt( 10 * m_n * .5)
            
# Vtensor0 = Vtensor_calc(phase_list)
# Vtensor_q0 = Vtensor_q0_calc(phase_list, Vtensor0, q0)

# sol = phase_shift_calc(phase_list, c0, q0, Vtensor_q0, lam)
# sol_name = ['d0','s_mat']
                        
# for i in range(len(sol)):
#     print('\n',sol_name[i],':')
#     if type(sol[i])==np.matrix:
#         print(round_n(sol[i]))
#     else:
#         print(round(sol[i],3))

In [12]:
# # Example

# phase_list = phase_dict[0]
# p_wave = [phase_name(*phase) for phase in phase_list]

# print('phases :',phase_list,'==',p_wave)

# lam = 2
# c0 = -45.39767
# q0 = sqrt( 10 * m_n * .5)
            
# Vtensor0_ = Vtensor_calc(phase_list)
# Vtensor_q0 = Vtensor_q0_calc(phase_list, Vtensor0_, q0)

# sol = phase_shift_calc(phase_list, c0, q0, Vtensor_q0, lam)

# sol_name = ['d0','d2','e0','s_mat']
# for i in range(len(sol)):
#     print('\n',sol_name[i],':')
#     if type(sol[i])==np.matrix:
#         print(round_n(sol[i]))
#     else:
#         print(round(sol[i],3))